In [6]:
import os
import json
import numpy as np
from astropy.table import Table, join, vstack, setdiff
import matplotlib.pyplot as plt

DRPVER = "1.2.0"

#sas = os.environ['SAS_BASE_DIR']

file = 'data_v1.2.0/lvmvis-drpall-1.2.0.fits'

t = Table.read(file)


t_sub = Table({'target': t['target'], 'count': np.ones(len(t), dtype=int)})
result = t_sub.group_by('target').groups.aggregate(np.sum)
result.pprint_all()

                           target                           count
----------------------------------------------------------- -----
                                                        0.0    11
                                                      30Dor    21
                                                      AV255     9
                                                      AV435    11
                                                   CIRCINUS     9
                                                     DEM103     8
                                                 ESO137-018     9
                                                 ESO223-009     9
                                                 ESO273-014     9
                                                 ESO274-001     1
                                                 ESO324-024     9
                                                 ESO325-011     9
                                                 ESO383-087     9
          

In [7]:
print(len(result))

197


In [32]:
result[result['count'] > 20].pprint_all()

                    target                     count
---------------------------------------------- -----
                                         30Dor    21
                                       FULLSKY  2387
                                       Fornax4    26
     Gaia DR3 6545825828948305536 100pt dither   100
                                   Gum_SW_limb   403
                                     Gum_shell   390
                                           LMC  2197
                                           MW1   348
                                           MW2  5136
                                       NGC3199    26
                                       NGC4945    60
                                       NGC5128    52
                                         NGC55    35
                                            --   316
                                         ORION   791
                                  ORION_SPARSE   245
                                         Orion

In [57]:
mask_ngc = (np.char.find(result['target'].astype(str), 'NGC') >= 0)
mask_ic =  (np.char.find(result['target'].astype(str), 'IC') >= 0)
mask_arp =  (np.char.find(result['target'].astype(str), 'Arp') >= 0)

result[mask_ngc | mask_ic | mask_arp].pprint_all()
print(np.sum(result['count'][mask_ngc | mask_ic | mask_arp]))

mask_lv = mask_ngc | mask_ic | mask_arp
mask_short  = np.char.find(result['target'].astype(str), 'short') >= 0
mask_wham = np.char.find(result['target'].astype(str), 'WHAM') >= 0
mask_moon = np.char.find(result['target'].astype(str), 'moon') >= 0



#mask_redo = ((result['count'] > 10) & (np.char.find(result['target'].astype(str), 'NGC') >= 0))

      target      count
----------------- -----
           IC1613    15
    IC1613_dither    17
           IC3104     9
           IC4390     9
           IC4402     9
           IC4662     9
           IC4710     9
           IC4721     8
           IC5078     9
           IC5152     9
           IC5201     9
           IC5240     9
           IC5267     9
          NGC1316     7
          NGC1365    18
         NGC1427a     3
  NGC1427a_dither     9
          NGC1978     3
           NGC253     5
       NGC253_PA0     2
      NGC253_PA15     2
          NGC2915     9
          NGC3109     9
          NGC3199    26
          NGC3621     9
          NGC4945    60
          NGC4988     6
          NGC5042     9
          NGC5068     9
          NGC5084     9
          NGC5102     9
          NGC5128    52
          NGC5236     9
          NGC5253     9
          NGC5398     9
            NGC55    35
          NGC5530     9
          NGC5556     9
          NGC5643     9
          NGC591

In [58]:
mask_redo = ((result['count'] > 10) & ~mask_lv) | mask_short | mask_wham | mask_moon
result[mask_redo].pprint_all()

                    target                     count
---------------------------------------------- -----
                                           0.0    11
                                         30Dor    21
                                         AV435    11
                                       FULLSKY  2387
                                       Fornax4    26
                                       G316.75    12
     Gaia DR3 6545825828948305536 100pt dither   100
                                   Gum_SW_limb   403
                                     Gum_shell   390
                                           LMC  2197
                                           M22    13
                                           MW1   348
                                           MW2  5136
                                           N44    12
                                    N44_dither    18
                                            --   316
                                         ORION

In [63]:
np.array(result[mask_redo]['target'])

array([b'0.0', b'30Dor', b'AV435', b'FULLSKY', b'Fornax4', b'G316.75',
       b'Gaia DR3 6545825828948305536 100pt dither', b'Gum_SW_limb',
       b'Gum_shell', b'LMC', b'M22', b'MW1', b'MW2', b'N44',
       b'N44_dither', b'', b'ORION', b'ORION_SPARSE', b'Orion',
       b'Pointing_accuracy Gaia DR3 5010284310919786496', b'RCW86',
       b'Rosette', b'SMC', b'SMCeast', b'SMCwest', b'THOR',
       b'WHAM_South_04', b'WHAM_south_01', b'WHAM_south_03',
       b'WHAM_south_04', b'WHAM_south_07', b'WHAM_south_08',
       b'lunar_eclipse_2025', b'moon', b'pointing accuracy test primary',
       b'pointing accuracy test secondary', b'shortexp_tile_id=1027966',
       b'shortexp_tile_id=1027966_0', b'shortexp_tile_id=1027966_1',
       b'shortexp_tile_id=1027966_2', b'shortexp_tile_id=1027966_3',
       b'shortexp_tile_id=1027966_4', b'shortexp_tile_id=1027966_5',
       b'shortexp_tile_id=1027966_6', b'shortexp_tile_id=1027966_7',
       b'shortexp_tile_id=1027966_8', b'shortexp_tile_id=10279

In [62]:
sum = np.sum(result['count'])
print(sum)
sum = np.sum(result['count'][mask_redo])
print(sum)
sum = np.sum(result['count'][~mask_redo])
print(sum)

15064
13967
1097


In [64]:
targets_redo = result['target'][~mask_redo]
print(targets_redo)

      target     
-----------------
            AV255
         CIRCINUS
           DEM103
       ESO137-018
       ESO223-009
       ESO273-014
       ESO274-001
       ESO324-024
       ESO325-011
       ESO383-087
              ...
          S308_P4
          S308_P5
          S308_P6
SV_NGC6822_dither
             SexA
         UGC10985
          UGCA348
          UGCA353
          UGCA394
              arc
      tile_id=999
Length = 140 rows


In [9]:
t

tileid,mjd,expnum,exptime,dpos,target,ra,dec,pa
int32,int32,int32,float64,uint8,bytes59,float64,float64,float32
11111,60179,3582,10.0,0,None,0.0,0.0,180.0
11111,60179,3583,180.0,0,None,281.74285,-3.94386,241.4142
1022427,60179,3584,60.0,0,THOR,281.74286,-3.94397,180.0
1022470,60179,3585,60.0,0,THOR,281.72183,-3.52334,180.0
1022428,60179,3586,60.0,0,THOR,282.07623,-3.29484,180.0
1022513,60179,3588,180.0,0,THOR,281.70084,-3.10272,241.3843
1022471,60179,3589,180.0,0,THOR,282.05517,-2.87426,241.4058
11111,60179,3590,900.0,0,None,271.04198,-30.03403,178.8122
11111,60179,3591,900.0,0,None,271.04197,-30.03395,178.827


In [65]:
mask = np.isin(t['target'], targets_redo.compressed())
t_selected = t[mask]

print(len(t_selected))
t_selected

1097


tileid,mjd,expnum,exptime,dpos,target,ra,dec,pa
int32,int32,int32,float64,uint8,bytes59,float64,float64,float32
11111,60179,3582,10.0,0,None,0.0,0.0,180.0
11111,60179,3583,180.0,0,None,281.74285,-3.94386,241.4142
11111,60179,3590,900.0,0,None,271.04198,-30.03403,178.8122
11111,60179,3591,900.0,0,None,271.04197,-30.03395,178.827
11111,60179,3592,900.0,0,None,271.04196,-30.03395,178.8477
11111,60179,3593,900.0,0,None,271.04197,-30.03391,178.8728
11111,60179,3594,60.0,0,Lagoon_2,271.5079,-24.0576,180.0
11111,60179,3595,900.0,0,Lagoon_2,271.50791,-24.05761,178.8919
11111,60179,3596,900.0,0,M20W28_P6,270.24998,-23.28332,178.9364


In [70]:
# write (your current cell is fine)
t_selected.write("drpall_redo.csv", format="ascii.csv",delimiter=",",overwrite=True)

# read back (no separator argument)
#t_selected_check = Table.read("drpall_redo.ecsv", format="ascii.ecsv")

In [ ]:
#
# Targets to DR20
#
